In [1]:
!pip install -q bitsandbytes peft datasets[audio] jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00


In [2]:
!pip install torch

In [3]:
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
from peft import PeftModel, PeftConfig
import torch
class SpeechRecognizer:
    def __init__(self, peft_model_id, language="malay", task="transcribe"):
        # Store the language and task
        self.language = language
        self.task = task

        # Load PeftConfig
        self.peft_model_id = peft_model_id
        peft_config = PeftConfig.from_pretrained(self.peft_model_id)

        # Load the model, tokenizer, processor, and other necessary components
        self.model = WhisperForConditionalGeneration.from_pretrained(
            peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
        )
        self.model = PeftModel.from_pretrained(self.model, self.peft_model_id)
        self.tokenizer = WhisperTokenizer.from_pretrained(peft_config.base_model_name_or_path, language=self.language, task=self.task)
        self.processor = WhisperProcessor.from_pretrained(peft_config.base_model_name_or_path, language=self.language, task=self.task)
        self.feature_extractor = self.processor.feature_extractor
        self.forced_decoder_ids = self.processor.get_decoder_prompt_ids(language=self.language, task=self.task)

        # Initialize the pipeline for ASR
        self.pipe = AutomaticSpeechRecognitionPipeline(
            model=self.model, tokenizer=self.tokenizer, feature_extractor=self.feature_extractor
        )

    def transcribe(self, audio_file):
        # Perform transcription with automatic mixed precision if CUDA is available
        if torch.cuda.is_available():
            with torch.cuda.amp.autocast():
                result = self.pipe(audio_file, generate_kwargs={"forced_decoder_ids": self.forced_decoder_ids}, max_new_tokens=255)
        else:
            result = self.pipe(audio_file, generate_kwargs={"forced_decoder_ids": self.forced_decoder_ids}, max_new_tokens=255)

        # Return the transcribed text
        return result["text"]

In [4]:
peft_model_id_1 = "clt013/whisper-large-v3-ft-malay-peft-epoch-20"
perf_model_id_2 = "clt013/whisper-small-ft-malay-peft-epoch-20"
whisper_large_v3_recognizer = SpeechRecognizer(peft_model_id=peft_model_id_1)
# whisper_small_recognizer = SpeechRecognizer(peft_model_id=perf_model_id_2)

adapter_config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

In [5]:
# save to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import time
from datasets import load_dataset, concatenate_datasets

def transcribe_and_compare(dataset_name,
                          #  whisper_small_recognizer,
                           whisper_large_v3_recognizer,
                           limit=10, limited=False):
    """
    Transcribe audio samples from the dataset using two different Whisper models, compare the transcriptions,
    and measure the processing time for each transcription.

    Args:
    - dataset_name (str): The name of the dataset to load.
    - whisper_small_recognizer: The recognizer object for the small Whisper model.
    - whisper_large_v3_recognizer: The recognizer object for the large Whisper model.
    - limit (int): The number of samples to process if 'limited' is True.
    - limited (bool): Whether to limit the number of samples processed.

    Returns:
    - df (DataFrame): A pandas DataFrame containing the reference sentence, transcriptions,
                      and processing times for the two Whisper models.
    """
    # Load the dataset
    dataset = load_dataset(dataset_name)

    # Concatenate train and test sets
    concatenated_dataset = concatenate_datasets([dataset["train"], dataset["test"]])

    # Prepare to store the results
    df_list = []

    # Loop through the dataset, limited by the specified amount if necessary
    for i in range(limit if limited else len(concatenated_dataset)):
        audio_sample = concatenated_dataset[i]['audio']
        samples = audio_sample['array']
        reference = concatenated_dataset[i]['sentence']

        # Transcribe using whisper_small_recognizer and measure processing time
        # start_time_small = time.time()
        # transcribed_1 = whisper_small_recognizer.transcribe(audio_sample)
        # end_time_small = time.time()
        # process_time_small = end_time_small - start_time_small

        # Transcribe using whisper_large_v3_recognizer and measure processing time
        start_time_large = time.time()
        # audio_sample = concatenated_dataset[i]['audio']
        transcribed_2 = whisper_large_v3_recognizer.transcribe(audio_sample)
        end_time_large = time.time()
        process_time_large = end_time_large - start_time_large

        # Append the results to the list, including processing times
        df_list.append(pd.DataFrame({
            'reference': [reference],
            # 'whisper_small_prediction': [transcribed_1],
            'whisper_large_v3_prediction': [transcribed_2],
            # 'whisper_small_process_time': [process_time_small],
            'whisper_large_v3_process_time': [process_time_large]
        }))

        print(f"Processed sample {i}: large model time = {process_time_large:.2f}s")

    # Combine the results into a single DataFrame
    df = pd.concat(df_list, ignore_index=True)

    return df

In [7]:
df_results = transcribe_and_compare("clt013/malay-speech-3k-rows-dataset_v2", whisper_large_v3_recognizer)
df_results

README.md:   0%|          | 0.00/460 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/216M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/59.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2544 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/604 [00:00<?, ? examples/s]

<ipython-input-3-bd40256c27b2>:37: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Processed sample 0: large model time = 4.77s
Processed sample 1: large model time = 2.65s
Processed sample 2: large model time = 2.64s
Processed sample 3: large model time = 1.15s
Processed sample 4: large model time = 1.75s
Processed sample 5: large model time = 2.14s
Processed sample 6: large model time = 2.51s
Processed sample 7: large model time = 2.81s
Processed sample 8: large model time = 2.16s


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processed sample 9: large model time = 1.27s
Processed sample 10: large model time = 1.53s
Processed sample 11: large model time = 1.72s
Processed sample 12: large model time = 3.30s
Processed sample 13: large model time = 0.99s
Processed sample 14: large model time = 0.87s
Processed sample 15: large model time = 3.67s
Processed sample 16: large model time = 1.55s
Processed sample 17: large model time = 0.89s
Processed sample 18: large model time = 1.53s
Processed sample 19: large model time = 1.51s
Processed sample 20: large model time = 4.46s
Processed sample 21: large model time = 0.93s
Processed sample 22: large model time = 0.86s
Processed sample 23: large model time = 1.47s
Processed sample 24: large model time = 3.03s
Processed sample 25: large model time = 3.08s
Processed sample 26: large model time = 3.84s
Processed sample 27: large model time = 2.62s
Processed sample 28: large model time = 1.34s
Processed sample 29: large model time = 4.30s
Processed sample 30: large model ti

,reference,whisper_large_v3_prediction,whisper_large_v3_process_time
0,hai weh,hai weh,4.771178
1,ah makanan bagi aku macam struggle kan,ah makanan bagi aku macam suggest kan,2.652175
2,makanan yang sedap kat Malaysia ni tu,makanan yang sedap kat Malaysia ni tu,2.643847
3,kenapa,kenapa,1.154674
4,dia punya dia punya,dia punya dia punya,1.750382
...,...,...,...
3143,okey aku pun tu je la makanan yang,okey aku pun itu je lah makanan yang,3.413080
3144,makanan kegemaran aku,makanan kegemaran aku,2.192711
3145,so,sob,1.103134
3146,jumpa lagi,jumpa lagi,1.519158


In [8]:
df_results.to_csv("peft_evaluate_large_epoch_20.csv", index=False)

!cp {"peft_evaluate_large_epoch_20.csv"} "/content/drive/MyDrive/"